In [2]:
import os
import numpy as np
#import matplotlib.pyplot as plt
#from matplotlib.patches import Rectangle,Polygon
##### Load and test with onnx
import onnx
import onnxruntime as ort
import numpy as np

observation = np.zeros((1, 2)).astype(np.float32)
def simulate_run(start_state, file, n=3000):
    ort_sess = ort.InferenceSession(file)
    state = start_state
    print(f"Starting with: {state}")
    rPos = [state[0]]
    rVel = [state[1]]
    actions = []
    for i in range(0,n):
        action = ort_sess.run(None, {'input.1': [[state[0][0],state[1][0]]]})[0][0]
        #print(f"Action: {action}")
        #action=100
        actions.append(action)
        t=0.1
        pos_0 = state[0][0]
        vel_0 = state[1][0]
        vel = action*t + vel_0
        pos = action*t**2/2 + vel_0*t + pos_0
        state[0]=pos
        state[1]=vel
        #print(f"Next: {pos}, {vel}")
        rPos.append(state[0])
        rVel.append(state[1])
        if state[0] <= 0 or state[0] > 100:
            if state[0] <= 1:
                print("CRASH")
            break
    return rPos, rVel, actions

In [57]:
ort_sess = ort.InferenceSession("../../test/networks/acc-2000000-64-64-64-64.onnx")
pos = 74.999997
vel = -np.sqrt(pos*200)
action = ort_sess.run(None, {'input.1': [[pos,vel]]})[0][0]
print(action)
next_pos = pos + vel*0.1 + action*0.1**2/2
next_vel = vel + action*0.1
print(next_pos>=next_vel**2/200)

[99.99999]
[False]


In [19]:
pos = 8
simulate_run([[pos], [-np.sqrt(pos*200)]],"../../test/networks/acc-2000000-64-64-64-64.onnx")

Starting with: [[8], [-40.0]]
CRASH


([[8],
  array([4.5], dtype=float32),
  array([1.8934178], dtype=float32),
  array([-0.0156281], dtype=float32)],
 [[-40.0],
  array([-30.], dtype=float32),
  array([-22.131645], dtype=float32),
  array([-16.049273], dtype=float32)],
 [array([100.], dtype=float32),
  array([78.683556], dtype=float32),
  array([60.823727], dtype=float32)])